In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import random

weather_csv = gpd.read_file("./기상청_종관기상관측_ASOS.csv", encoding='cp949')
accident_csv = gpd.read_file("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
length = len(accident_csv)


In [14]:
import requests
import time

new_idx = []
new_x = []
new_y = []
print(length)
for idx in range(0, length):  
    if(str(accident_csv['loc_info_y'][idx]) != ''):
        method = "GET"
        url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        params = {'x' : str(accident_csv['loc_info_x'][idx]), 
                  'y' : str(accident_csv['loc_info_y'][idx]),
                  'input_coord' : 'TM',
                  'output_coord' : 'WGS84',
                  'Content-Type' : 'application/x-www-form-urlencoded'}
        header = {'authorization': 'KakaoAK 2bf0aa36b38bfed87150a8fd1246ee39'}

        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()

        x = tokens['documents'][0]['x']
        y = tokens['documents'][0]['y'] - 0.453

        new_idx.append(accident_csv['resc_reprt_no'])
        new_x.append(x)
        new_y.append(y)
        time.sleep(0.5)
    else: #has no coordinate
        print(idx)

1547
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [15]:
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=10)
m.save('map.html')

In [19]:
import geopandas as gpd
import pandas as pd
import matplotlib as plt

si = gpd.read_file('./CTPRVN_20220324/ctp_rvn.shp', encoding = 'euc-kr')
si = si.to_crs(epsg=4326)
dong = gpd.read_file("./EMD_20220324/emd.shp", encoding = 'euc-kr')
dong = dong.to_crs(epsg=4326)
si = si[si['CTP_ENG_NM'] == 'Jeju-do']

dropList = []
for i in range(0, len(dong)):
    if(si['geometry'].intersects(dong['geometry'][i]).bool() == False):
        dropList.append(i)

dong = dong.drop(dropList)

east_list = ["구좌읍", "우도면", "표선면", "성산읍"]
west_list = ["한림읍", "한경면", "대정읍"]
north_list = ["애월읍", "조천읍", "해안동", "봉개동", "회천동", "영평동", "연동", "노형동", "용강동", "오라이동", "오등동", "아라일동", "아라이동", "화북일동", "화북이동", "도련일동", "도련이동", "삼양일동", "삼양이동", "삼양삼동", "일도이동","월평일동", "이도이동", "오라일동", "오라이동", "오라삼동", "건입동", "도남동", "용담일동", "용담이동", "용담삼동", "도두일동", "도두이동", "삼도일동", "삼도이동", "일도일동", "이도일동", "이도이동", "이호일동", "이호이동", "외도일동", "외도이동", "내도동", "도평동", "추자면"]
south_list = ["안덕면", "남원읍", "상예동", "하예동", "색달동", "중문동", "대포동", "회수동", "하원동", "도순동", "영남동", "강정동", "법환동", "서호동", "호근동", "서홍동", "동홍동", "서귀동", "토평동", "상효동", "보목동", "하효동", "신효동", "월평이동"]

dong['LOCATION'] = ""
for index, r in dong.iterrows():
    
    lat = r['geometry'].centroid.y
    lon = r['geometry'].centroid.x
    
    if(r['EMD_KOR_NM'] == "월평동"and lat == 33.43469125180385 and lon == 126.58289458250032):
        dong.loc[index, 'EMD_KOR_NM'] = "월평일동"
        r['EMD_KOR_NM'] = "월평일동"
    elif(lat == 33.24435422745148 and lon == 126.46377031438104):
        dong.loc[index, 'EMD_KOR_NM'] = "월평이동"
        r['EMD_KOR_NM'] = "월평이동"
        
    if(r['EMD_KOR_NM'] in east_list):
        dong.loc[index, 'LOCATION'] = "EAST"
    elif(r['EMD_KOR_NM'] in west_list):
        dong.loc[index, 'LOCATION'] = "WEST"
    elif(r['EMD_KOR_NM'] in north_list):
        dong.loc[index, 'LOCATION'] = "NORTH"
    elif(r['EMD_KOR_NM'] in south_list):
        dong.loc[index, 'LOCATION'] = "SOUTH"

continents = dong.dissolve(by='LOCATION', aggfunc='sum')
jeju_east = continents['geometry']['EAST']
jeju_west = continents['geometry']['WEST']
jeju_north = continents['geometry']['NORTH']
jeju_south = continents['geometry']['SOUTH']

In [20]:
for _, r in dong.iterrows():

    
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()

    if(r['LOCATION'] == "EAST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green'})
    elif(r['LOCATION'] == "WEST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'blue'})
    elif(r['LOCATION'] == "NORTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'red'})
    elif(r['LOCATION'] == "SOUTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange'})
    else:
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'black'})

    lat = r['geometry'].centroid.y
    lon = r['geometry'].centroid.x
    #folium.Marker(location=[lat, lon],popup='{}'.format(r['EMD_KOR_NM'])).add_to(m)


    geo_j.add_to(m)
m

In [28]:
import numpy as np
from folium import plugins
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=10)

direction_list = [0, 0, 0, 0]
direction_name = ['east', 'west', 'north', 'south']
distance_list = [0, 0, 0, 0]
color_list = ['green', 'blue', 'red', 'yellow']
section_list = [0, 0, 0, 0]

direction = []

fg = folium.FeatureGroup(name = 'groups')
m.add_child(fg)

for i in range(0, len(direction_list)):
    direction_list[i] = plugins.FeatureGroupSubGroup(fg, direction_name[i])
    m.add_child(direction_list[i])

for i in range(0, len(new_x)):
    point = Point(new_x[i], new_y[i])
    distance_list[0] = point.distance(jeju_east)
    distance_list[1] = point.distance(jeju_west)
    distance_list[2] = point.distance(jeju_north)
    distance_list[3] = point.distance(jeju_south)

    nearest_section = np.argmin(distance_list) #get the nearest section if intersects, distance is 0
    section_list[nearest_section] += 1
    folium.Circle(radius=3,location=[new_y[i], new_x[i]], color=color_list[nearest_section], fill=True,).add_to(direction_list[nearest_section])
    
    direction.append(direction_name[nearest_section])
    
print("Total Data : " + str(len(new_x)))
print("EAST Accident Count : " + str(section_list[0]))
print("WEST Accident Count : " + str(section_list[1]))
print("SOUTH Accident Count : " + str(section_list[2]))
print("NORTH Accident Count : " + str(section_list[3]))

direction.append(" ")
folium.LayerControl(collapsed=False).add_to(m)
m.save("section.html")
m

Total Data : 1546
EAST Accident Count : 277
WEST Accident Count : 226
SOUTH Accident Count : 704
NORTH Accident Count : 339


In [30]:
df = pd.read_csv("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
df['direction'] = direction
df.to_csv("out.csv", encoding = 'euc-kr')
